In [1]:
%pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 8.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.1
    Uninstalling scipy-1.16.1:
      Successfully uninstalled scipy-1.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from gensim.models.ldamodel import LdaModel

# Load the LDA model
loaded_lda_model = LdaModel.load('/content/drive/MyDrive/Colab Notebooks/alex/models/lda_10_topics/lda_model_2.gensim')

# Now you can access information from the loaded model.
# For example, you can print the topics:
print("Topics:")
for topic_id, topic in loaded_lda_model.print_topics():
    print(f"Topic {topic_id}: {topic}")

# You can also access the dictionary (id2word)
print("\nDictionary:")
print(loaded_lda_model.id2word)

# And the number of topics
print("\nNumber of topics:")
print(loaded_lda_model.num_topics)

Topics:
Topic 0: 0.025*"football" + 0.016*"game" + 0.015*"season" + 0.014*"state" + 0.011*"usc" + 0.011*"bowl" + 0.010*"quarterback" + 0.009*"college" + 0.009*"coach" + 0.009*"yard"
Topic 1: 0.027*"woman" + 0.019*"coach" + 0.017*"school" + 0.017*"year" + 0.015*"team" + 0.014*"basketball" + 0.012*"season" + 0.011*"player" + 0.011*"high" + 0.011*"college"
Topic 2: 0.022*"gopher" + 0.018*"game" + 0.017*"goal" + 0.016*"hockey" + 0.015*"minnesota" + 0.012*"season" + 0.012*"team" + 0.009*"first" + 0.007*"two" + 0.007*"four"
Topic 3: 0.010*"sport" + 0.008*"game" + 0.006*"one" + 0.006*"fan" + 0.006*"basketball" + 0.006*"time" + 0.005*"like" + 0.005*"people" + 0.005*"would" + 0.004*"day"
Topic 4: 0.014*"said" + 0.013*"college" + 0.012*"sport" + 0.012*"ncaa" + 0.011*"school" + 0.010*"athlete" + 0.009*"state" + 0.007*"would" + 0.007*"university" + 0.006*"football"
Topic 5: 0.009*"u" + 0.008*"first" + 0.008*"championship" + 0.008*"world" + 0.007*"team" + 0.007*"league" + 0.006*"olympic" + 0.006*"w

In [3]:
from gensim import corpora

# Load the corpus from the .mm file
corpus = corpora.MmCorpus('/content/drive/MyDrive/Colab Notebooks/alex/models/dictionary_and_corpus/corpus.mm')

print("Corpus loaded successfully.")
# You can optionally print the corpus to inspect its structure
# print(list(corpus))

Corpus loaded successfully.


In [4]:
import pandas as pd

# Load the tokenized data from the Excel file
try:
    tokenized_data_df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/alex/updated_ tokenized_newspaper_data.xlsx')
    print("Excel file loaded successfully.")
    # Assuming the original text is in a column named 'text' or similar.
    # Please adjust the column name if it's different in your Excel file.
    if 'Main_text_tokenized' in tokenized_data_df.columns:
        texts = tokenized_data_df['Main_text_tokenized'].tolist()
        print(f"Extracted {len(texts)} documents.")
    else:
        texts = None
        print("Could not find a 'text' column in the Excel file. Please make sure the column containing original texts is named 'text' or update the code.")

except FileNotFoundError:
    print("Error: The file was not found at the specified path.")
except Exception as e:
    print(f"An error occurred while reading the Excel file: {e}")

Excel file loaded successfully.
Extracted 149703 documents.


In [5]:
texts

Output hidden; open in https://colab.research.google.com to view.

In [6]:
#TFinding the dominant topic in each sentence

def format_topics_sentences(ldamodel=loaded_lda_model, corpus=corpus, texts=texts): # Updated to use loaded_lda_model and corpus, and texts=None
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = pd.concat([sent_topics_df, pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]).to_frame().T], ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

     # Add original text to the end of the output
    if texts is not None: # Added check if texts is not None
        contents = pd.Series(texts)
        sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


# Assuming texts is available as data_words or you provide another list/Series of texts
# df_topic_sents_keywords = format_topics_sentences(ldamodel=loaded_lda_model, corpus=corpus, texts=texts) # Updated to use loaded_lda_model and corpus, commented out as data_words is undefined
# To run this cell, define `data_words` with your text data before calling the function.
# For example:
# data_words = ["your document text 1", "your document text 2", ...]
# Then uncomment the line below:
df_topic_sents_keywords = format_topics_sentences(ldamodel=loaded_lda_model, corpus=corpus, texts=texts)

# Format
if 'df_topic_sents_keywords' in locals(): # Added check if df_topic_sents_keywords is defined
    df_dominant_topic = df_topic_sents_keywords.reset_index()
    df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
    # Show
    display(df_dominant_topic.head(20)) # Use display for better formatting

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,3,0.362,"sport, game, one, fan, basketball, time, like,...","['14', 'year', 'coaching', '39', 'year', 'livi..."
1,1,1,0.3167,"woman, coach, school, year, team, basketball, ...","['see', 'sidebar', '02c', 'ruston', 'la', 'din..."
2,2,6,0.4494,"said, player, team, like, coach, game, going, ...","['gave', 'garden', 'exactly', 'missed', 'lonel..."
3,3,6,0.4017,"said, player, team, like, coach, game, going, ...","['indianapolis', 'ten', 'year', 'ago', 'chuck'..."
4,4,5,0.3704,"u, first, championship, world, team, league, o...","['robinson', 'talk', 'ranger', 'stanley', 'cup..."
5,5,7,0.3365,"v, pm, game, state, 1, 2, 3, 2024, 7, 4","['usc', 'conference', 'pacific10', 'coach', 'm..."
6,6,5,0.4352,"u, first, championship, world, team, league, o...","['ranger', 'signed', 'center', 'michael', 'nyl..."
7,7,5,0.5079,"u, first, championship, world, team, league, o...","['gavitt', 'five', 'others', 'elected', 'hall'..."
8,8,6,0.3956,"said, player, team, like, coach, game, going, ...","['almost', 'mythical', 'element', 'lsu', 'seni..."
9,9,3,0.3996,"sport, game, one, fan, basketball, time, like,...","['ncaa', 'woman', 'jim', 'izard', 'diana', 'vi..."


In [7]:
# Display the first few rows of the dataframe to see the structure
display(df_dominant_topic.head())

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,3,0.362,"sport, game, one, fan, basketball, time, like,...","['14', 'year', 'coaching', '39', 'year', 'livi..."
1,1,1,0.3167,"woman, coach, school, year, team, basketball, ...","['see', 'sidebar', '02c', 'ruston', 'la', 'din..."
2,2,6,0.4494,"said, player, team, like, coach, game, going, ...","['gave', 'garden', 'exactly', 'missed', 'lonel..."
3,3,6,0.4017,"said, player, team, like, coach, game, going, ...","['indianapolis', 'ten', 'year', 'ago', 'chuck'..."
4,4,5,0.3704,"u, first, championship, world, team, league, o...","['robinson', 'talk', 'ranger', 'stanley', 'cup..."


In [8]:
# Sort documents within each topic by percentage contribution in descending order
sorted_documents_by_topic = df_dominant_topic.sort_values(by=['Dominant_Topic', 'Topic_Perc_Contrib'], ascending=[True, False])

# Display the sorted document indexes for each topic
print("Document Indexes for Each Topic (Sorted by Percentage Contribution):")
for topic_num in sorted_documents_by_topic['Dominant_Topic'].unique():
    print(f"\nTopic {topic_num}:")
    # Get documents for the current topic and select only the 'Document_No' column
    topic_documents = sorted_documents_by_topic[sorted_documents_by_topic['Dominant_Topic'] == topic_num]['Document_No']
    # Display the document indexes (first 20 as an example)
    print(topic_documents.tolist()[:20])

Document Indexes for Each Topic (Sorted by Percentage Contribution):

Topic 0:
[26776, 51816, 57373, 79266, 117019, 97018, 116896, 79168, 37623, 3311, 53884, 116912, 117198, 117223, 115513, 11376, 94703, 63912, 87497, 130450]

Topic 1:
[3163, 64700, 43751, 71089, 105769, 3155, 67939, 66229, 70296, 69196, 83855, 145976, 85107, 57636, 34546, 136344, 45644, 62821, 32953, 78365]

Topic 2:
[22572, 5037, 95156, 137523, 57253, 4732, 135546, 99549, 24103, 12479, 92804, 83747, 106105, 142444, 67145, 122383, 129260, 108770, 148023, 103497]

Topic 3:
[38825, 144759, 95222, 36898, 136076, 80861, 49210, 104705, 73672, 80819, 110460, 24353, 24572, 23050, 37588, 46842, 110461, 135036, 94540, 133532]

Topic 4:
[69832, 70792, 14390, 132924, 32177, 25427, 124934, 63946, 1145, 127699, 32748, 70785, 92629, 90292, 146373, 112461, 21511, 138582, 91596, 148124]

Topic 5:
[105809, 37076, 60341, 53263, 21267, 54726, 60336, 82108, 126903, 30536, 85339, 15065, 48027, 83131, 91412, 126425, 9912, 12611, 126432, 21

In [9]:
# Get the total number of documents for each dominant topic
topic_counts = df_dominant_topic['Dominant_Topic'].value_counts().sort_index()

print("Total number of documents for each topic:")
display(topic_counts)

Total number of documents for each topic:


,count
Dominant_Topic,
0,10514
1,11224
2,5846
3,14945
4,16797
5,14196
6,38546
7,4877
8,14599


In [10]:
# Get the top 100 documents for each topic
top_100_documents_per_topic = pd.DataFrame()

for topic_num in sorted_documents_by_topic['Dominant_Topic'].unique():
    # Get the top 100 documents for the current topic
    top_100 = sorted_documents_by_topic[sorted_documents_by_topic['Dominant_Topic'] == topic_num].head(100)
    # Append to the results DataFrame
    top_100_documents_per_topic = pd.concat([top_100_documents_per_topic, top_100], ignore_index=True)

# Display the top 100 documents for each topic
print("Top 100 Documents for Each Topic (Sorted by Percentage Contribution):")

# Iterate through topics and display the top documents for each
for topic_num in top_100_documents_per_topic['Dominant_Topic'].unique():
    print(f"\n--- Topic {topic_num} ---")
    display(top_100_documents_per_topic[top_100_documents_per_topic['Dominant_Topic'] == topic_num])

Top 100 Documents for Each Topic (Sorted by Percentage Contribution):

--- Topic 0 ---


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,26776,0,0.9382,"football, game, season, state, usc, bowl, quar...","['idol', 'reign', 'cbs', 'held', 'network', 'l..."
1,51816,0,0.9129,"football, game, season, state, usc, bowl, quar...","['first', 'time', 'three', 'year', 'kansa', 'c..."
2,57373,0,0.9098,"football, game, season, state, usc, bowl, quar...","['fox', 'sport', 'president', 'ed', 'goren', '..."
3,79266,0,0.9058,"football, game, season, state, usc, bowl, quar...","['brandconscious', 'world', 'network', 'one', ..."
4,117019,0,0.9008,"football, game, season, state, usc, bowl, quar...","['medium', 'obsesses', 'thing', 'donald', 'tru..."
...,...,...,...,...,...
95,58919,0,0.7603,"football, game, season, state, usc, bowl, quar...","['two', 'small', 'step', 'display', 'today', '..."
96,104120,0,0.7595,"football, game, season, state, usc, bowl, quar...","['breanna', 'stewart', 'said', 'wanted', 'win'..."
97,60756,0,0.7589,"football, game, season, state, usc, bowl, quar...","['offensive', 'tackle', 'mark', 'ortmann', 'th..."
98,20053,0,0.7588,"football, game, season, state, usc, bowl, quar...","['chris', 'webber', 'high', 'school', 'refused..."



--- Topic 1 ---


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
100,3163,1,0.9876,"woman, coach, school, year, team, basketball, ...","['wrapped', 'regular', 'season', 'second', 'ma..."
101,64700,1,0.9648,"woman, coach, school, year, team, basketball, ...","['one', 'day', 'nick', 'pita', 'return', 'wind..."
102,43751,1,0.9494,"woman, coach, school, year, team, basketball, ...","['duke', 'athletic', 'director', 'joe', 'allev..."
103,71089,1,0.9488,"woman, coach, school, year, team, basketball, ...","['tipoff', '6', 'pm', 'thursday', 'prudential'..."
104,105769,1,0.9473,"woman, coach, school, year, team, basketball, ...","['question', 'arrives', 'almost', 'daily', 'ko..."
...,...,...,...,...,...
195,79133,1,0.7857,"woman, coach, school, year, team, basketball, ...","['statistic', 'drexel', 'derrick', 'thomas', '..."
196,23798,1,0.7856,"woman, coach, school, year, team, basketball, ...","['41', 'minute', 'pittsburgh', 'defensive', 'a..."
197,2532,1,0.7855,"woman, coach, school, year, team, basketball, ...","['sport', 'woman', 'year', 'woman', 'ncaa', 'f..."
198,20319,1,0.7851,"woman, coach, school, year, team, basketball, ...","['start', 'march', 'madness', 'always', 'bring..."



--- Topic 2 ---


...

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
700,132029,7,0.9915,"v, pm, game, state, 1, 2, 3, 2024, 7, 4","['supply', 'chain', 'problem', 'slowed', 'manu..."
701,143634,7,0.9863,"v, pm, game, state, 1, 2, 3, 2024, 7, 4","['c', 'onnor', 'bedard', '19yearold', 'blackha..."
702,110990,7,0.9855,"v, pm, game, state, 1, 2, 3, 2024, 7, 4","['matter', 'many', 'plantain', 'chip', 'lentil..."
703,52557,7,0.9737,"v, pm, game, state, 1, 2, 3, 2024, 7, 4","['usc', 'coach', 'trainer', 'kept', 'cj', 'gab..."
704,71952,7,0.969,"v, pm, game, state, 1, 2, 3, 2024, 7, 4","['10', '1', '2', 'year', 'job', 'western', 'ke..."
...,...,...,...,...,...
795,134252,7,0.8228,"v, pm, game, state, 1, 2, 3, 2024, 7, 4","['beginning', 'season', 'spotlight', 'fell', '..."
796,110957,7,0.8207,"v, pm, game, state, 1, 2, 3, 2024, 7, 4","['trophy', 'waiting', 'winner', 'northeastern'..."
797,149341,7,0.8207,"v, pm, game, state, 1, 2, 3, 2024, 7, 4","['one', 'greatest', 'player', 'woman', 'basket..."
798,87141,7,0.8201,"v, pm, game, state, 1, 2, 3, 2024, 7, 4","['offensive', 'efficiency', 'sorry', 'cam', 'b..."



--- Topic 8 ---


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
800,118679,8,0.9952,"tournament, team, state, game, ncaa, big, four...","['richard', 'pitino', 'recruiting', 'picking',..."
801,71163,8,0.989,"tournament, team, state, game, ncaa, big, four...","['bruin', 'notebook', 'wilmington', 'bruin', '..."
802,66419,8,0.9887,"tournament, team, state, game, ncaa, big, four...","['olympic', 'note', 'losing', 'home', 'depot',..."
803,71087,8,0.9852,"tournament, team, state, game, ncaa, big, four...","['boston', 'basketball', 'term', 'saturday', '..."
804,10637,8,0.9836,"tournament, team, state, game, ncaa, big, four...","['jake', 'serfass', 'crossroad', 'sort', 'earl..."
...,...,...,...,...,...
895,146601,8,0.8901,"tournament, team, state, game, ncaa, big, four...","['raleigh', 'nc', 'cooper', 'flagg', 'projecte..."
896,147331,8,0.8901,"tournament, team, state, game, ncaa, big, four...","['gopher', 'fired', 'men', 'basketball', 'coac..."
897,50102,8,0.8875,"tournament, team, state, game, ncaa, big, four...","['correction', 'chart', 'sunday', 'sport', 'se..."
898,50103,8,0.8875,"tournament, team, state, game, ncaa, big, four...","['regarding', 'illiniwek', 'fight', 'get', 'tw..."



--- Topic 9 ---


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
900,21242,9,0.985,"point, game, second, first, rebound, lead, sho...","['college', 'basketball', 'southwest', 'missou..."
901,49485,9,0.9548,"point, game, second, first, rebound, lead, sho...","['orlando', 'fla', 'rich', 'history', 'enough'..."
902,23631,9,0.9494,"point, game, second, first, rebound, lead, sho...","['last', 'row', 'bold', 'name', 'justin', 'spr..."
903,44342,9,0.9407,"point, game, second, first, rebound, lead, sho...","['larry', 'brown', 'stephon', 'marbury', 'feas..."
904,41985,9,0.9375,"point, game, second, first, rebound, lead, sho...","['scene', 'beamon', 'student', 'life', 'center..."
...,...,...,...,...,...
995,118624,9,0.8686,"point, game, second, first, rebound, lead, sho...","['last', 'week', 'minute', 'client', 'received..."
996,102793,9,0.8682,"point, game, second, first, rebound, lead, sho...","['kelsey', 'mitchell', 'sophomore', 'guard', '..."
997,148072,9,0.8681,"point, game, second, first, rebound, lead, sho...","['march', 'often', 'thought', 'term', 'spring'..."
998,13990,9,0.8679,"point, game, second, first, rebound, lead, sho...","['college', 'basketball', 'recent', 'morning',..."


In [11]:
# Define the path to save the Excel file in your Google Drive
excel_file_path = '/content/drive/MyDrive/Colab Notebooks/alex/top_100_articles/top_100_documents_per_topic_for_10_topics.xlsx'

# Save the DataFrame to an Excel file
try:
    top_100_documents_per_topic.to_excel(excel_file_path, index=False)
    print(f"Successfully saved the top 100 documents per topic to: {excel_file_path}")
except Exception as e:
    print(f"An error occurred while saving the Excel file: {e}")

Successfully saved the top 100 documents per topic to: /content/drive/MyDrive/Colab Notebooks/alex/top_100_articles/top_100_documents_per_topic_for_10_topics.xlsx


##### **© 2024–2025 MD Rafsun Sheikh**
##### **Licensed under the Apache License, Version 2.0.**